## 18 메모리 절약 모드

### 18.1 필요 없는 미분값 삭제

#### 이전 코드


In [1]:
class Variable:
  def __init__(self, data) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None
    #세대 수 표현
    self.generation = 0
  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1
  
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key = lambda x: x.generation)

    add_func(self.creator)
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator)

  def cleargrad(self):
    self.grad = None

In [2]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [3]:
import weakref
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    # 별표를 붙여 언팩
    ys = self.forward(*xs)
    # 튜플이 아닌 경우 추가 지원
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    # x 중에 가장 큰 세대 다음이니까
    self.generation = max([x.generation for x in inputs])
    for output in outputs :
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [4]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [5]:
def add(x0, x1):
  return Add()(x0,x1)

In [6]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [7]:
def square(x):
  return Square()(x)

In [9]:
import numpy as np
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0,x1)
y = add(x0,t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


+ 사실 중간에 있는 미분 값은 그렇게 중요하지 않다. 그래서 중간 미분 값을 제거하는 모드를 추가하겠다.

#### 변경한 코드

In [10]:
class Variable:
  def __init__(self, data) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None
    #세대 수 표현
    self.generation = 0
  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1
  
  def backward(self, retain_grad = False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key = lambda x: x.generation)

    add_func(self.creator)
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator)
      # 중간 미분값 삭제
      if not retain_grad:
        for y in f.outputs:
          y().grad = None
  def cleargrad(self):
    self.grad = None

In [11]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0,x1)
y = add(x0,t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


### 18.2 Function 클래스 복습

In [12]:
import weakref
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    # 별표를 붙여 언팩
    ys = self.forward(*xs)
    # 튜플이 아닌 경우 추가 지원
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    # x 중에 가장 큰 세대 다음이니까
    self.generation = max([x.generation for x in inputs])
    for output in outputs :
      output.set_creator(self)
    # 우리는 순전파를 했을 때 무조건 인스턴스 변수를 참조한다. 이것은 역전파 계산시 사용 되지만 때로는 미분값이 필요없는 경우도 있다. 이때는 사용하지 않으면 메모리 절약
    self.inputs = inputs
    self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

### 18.3 Config 클래스를 활용한 모드 전환
+ 역전파를 하지 않으면 모든 input 관련 코드를 실행시키지 않음

In [25]:
class Config:
  enable_backprop =True

In [26]:
import weakref
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs :
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [27]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [28]:
def add(x0, x1):
  return Add()(x0,x1)

In [29]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [30]:
def square(x):
  return Square()(x)

### 18.4 모드 전환

In [36]:
Config.enable_backprop = True
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()
print(x.grad)


[[[8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  ...
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]]

 [[8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  ...
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]]

 [[8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  ...
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]]

 ...

 [[8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  ...
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]]

 [[8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  ...
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]]

 [[8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  ...
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]
  [8. 8. 8. ... 8. 8. 8.]]]


In [37]:

Config.enable_backprop = False
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
print(x.grad)

None


### 18.5 with 문을 활용한 모드 전환

In [39]:
import contextlib

@contextlib.contextmanager
def config_test():
  print('start')
  try : 
    yield

  finally:
    print('done')

with config_test():
  print('process...')

start
process...
done


In [42]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config,name)
  setattr(Config, name, value)
  try : 
    yield

  finally:
    setattr(Config, name, old_value)


In [45]:
with using_config('enable_backprop',False):
  x = Variable(np.array(2.0))
  y = square(x)



In [44]:
def no_grad():
  return using_config('enable_backprop',False)

with no_grad():
  x = Variable(np.array(2.0))
  y = square(x)